### Homework 03

Name: Yining Liu  
Github Username: Lynzz1701    
USC ID: 6168529797 

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats
import os

In [2]:
AReM = '../data/AReM'
activities = os.listdir(AReM)
activities.sort()
idx = 1

for activity in activities:
    if (os.path.splitext(activity)[1] == '') and (activity != '.DS_Store'):
        instances = os.listdir(os.path.join(AReM,activity))
        instances.sort(key = lambda x:int(x.split('.')[0].split('set')[1]))
        
        for csv in instances:
            df_tmp = pd.read_csv(os.path.join(AReM,activity,csv), skiprows = 5,
                            names=['time','avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23'])
            df_tmp['activity'] = activity
            df_tmp['instance'] = idx
            idx += 1
            #if df_tmp.shape[0] != 480:
                #print(idx)
            if csv == instances[0] and activity == 'bending1':
                df = df_tmp
            else:
                df = pd.concat([df,df_tmp])

df
#os.makedirs('../data', exist_ok=True)  
#df.to_csv('../data/out.csv') 

#df_mul = df.set_index(['activity','instance'])
#df_mul
#df_mul.loc['bending1'].loc[2]

,time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,activity,instance
0,0,39.25,0.43,22.75,0.43,33.75,1.30,bending1,1
1,250,39.25,0.43,23.00,0.00,33.00,0.00,bending1,1
2,500,39.25,0.43,23.25,0.43,33.00,0.00,bending1,1
3,750,39.50,0.50,23.00,0.71,33.00,0.00,bending1,1
4,1000,39.50,0.50,24.00,0.00,33.00,0.00,bending1,1
...,...,...,...,...,...,...,...,...,...
475,118750,34.50,6.18,9.00,3.56,12.67,4.19,walking,88
476,119000,25.75,6.02,13.75,2.05,16.00,1.58,walking,88
477,119250,31.50,3.35,10.25,5.12,16.25,2.95,walking,88
478,119500,33.75,2.77,14.00,3.24,13.75,0.43,walking,88


#### 1(b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data.

In [3]:
df_idx = df
df_idx = df_idx.set_index('instance')
test_list = [1,2,8,9,14,15,16,29,30,31,44,45,46,59,60,61,74,75,76]
train_list = [i for i in range(1,89) if i not in test_list]

#19 test instances 9,120 rows
test_data = df_idx.loc[test_list]

#69 test instances 33,119 rows
train_data = df_idx.loc[train_list]

In [4]:
test_data

,time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,activity
instance,,,,,,,,
1,0,39.25,0.43,22.75,0.43,33.75,1.30,bending1
1,250,39.25,0.43,23.00,0.00,33.00,0.00,bending1
1,500,39.25,0.43,23.25,0.43,33.00,0.00,bending1
1,750,39.50,0.50,23.00,0.71,33.00,0.00,bending1
1,1000,39.50,0.50,24.00,0.00,33.00,0.00,bending1
...,...,...,...,...,...,...,...,...
76,118750,36.00,2.45,17.00,5.10,20.50,0.87,walking
76,119000,34.33,1.89,15.00,2.45,17.00,2.12,walking
76,119250,33.00,7.35,14.60,3.14,13.00,5.70,walking


In [5]:
train_data

,time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,activity
instance,,,,,,,,
3,0,42.00,0.71,21.25,0.43,30.00,0.00,bending1
3,250,41.50,0.50,20.25,1.48,31.25,1.09,bending1
3,500,41.50,0.50,14.25,1.92,33.00,0.00,bending1
3,750,40.75,0.83,15.75,0.43,33.00,0.00,bending1
3,1000,40.00,0.71,20.00,2.74,32.75,0.43,bending1
...,...,...,...,...,...,...,...,...
88,118750,34.50,6.18,9.00,3.56,12.67,4.19,walking
88,119000,25.75,6.02,13.75,2.05,16.00,1.58,walking
88,119250,31.50,3.35,10.25,5.12,16.25,2.95,walking


#### 1(c)i  Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc).

- Universal ones
    - maximum, minimum, length, mean, std, quantiles
    - energy, sum of changes
    - duplicates
    - peaks
- Morn aimed/focused ones for time series data
    - entropy
    - kurtosis
    - seasonal & nonseasonal cycles
    - timewise trends
    - pulses and steps
    - autocorrelation

#### 1(c)ii Extract the time-domain features minimum, maximum, mean, median, stan- dard deviation, first quartile, and third quartile for all of the 6 time series in each instance.

In [6]:
#part1 - raw features

info_df = pd.DataFrame(columns = ['min_1','max_1','mean_1','median_1','std_1','1st_quart_1','3rd_quart_1',
                                 'min_2','max_2','mean_2','median_2','std_2','1st_quart_2','3rd_quart_2',
                                 'min_3','max_3','mean_3','median_3','std_3','1st_quart_3','3rd_quart_3',
                                 'min_4','max_4','mean_4','median_4','std_4','1st_quart_4','3rd_quart_4',
                                 'min_5','max_5','mean_5','median_5','std_5','1st_quart_5','3rd_quart_5',
                                 'min_6','max_6','mean_6','median_6','std_6','1st_quart_6','3rd_quart_6',])

#df_idx = df.set_index('instance')

for i in range(1,89):
    row_tmp = []
    for j in range(1,7):
        slice_tmp = [df_idx.loc[i].iloc[:,j].min(), df_idx.loc[i].iloc[:,j].max(), df_idx.loc[i].iloc[:,j].mean(),
                     df_idx.loc[i].iloc[:,j].median(), df_idx.loc[i].iloc[:,j].std(), 
                     df_idx.loc[i].iloc[:,j].quantile(.25),df_idx.loc[i].iloc[:,j].quantile(.75)]
        row_tmp.extend(slice_tmp)
    info_df.loc[i] = row_tmp
    
info_df

,min_1,max_1,mean_1,median_1,std_1,1st_quart_1,3rd_quart_1,min_2,max_2,mean_2,...,std_5,1st_quart_5,3rd_quart_5,min_6,max_6,mean_6,median_6,std_6,1st_quart_6,3rd_quart_6
1,37.25,45.00,40.624792,40.50,1.476967,39.25,42.0000,0.0,1.30,0.358604,...,2.188449,33.0000,36.00,0.00,1.92,0.570583,0.43,0.582915,0.0000,1.3000
2,38.00,45.67,42.812812,42.50,1.435550,42.00,43.6700,0.0,1.22,0.372438,...,1.995255,32.0000,34.50,0.00,3.11,0.571083,0.43,0.601010,0.0000,1.3000
3,35.00,47.40,43.954500,44.33,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,1.999604,35.3625,36.50,0.00,1.79,0.493292,0.43,0.513506,0.0000,0.9400
4,33.00,47.75,42.179813,43.50,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,3.849448,30.4575,36.33,0.00,2.18,0.613521,0.50,0.524317,0.0000,1.0000
5,33.00,45.75,41.678063,41.75,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,2.411026,28.4575,31.25,0.00,1.79,0.383292,0.43,0.389164,0.0000,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,19.50,45.33,33.586875,34.25,4.650935,30.25,37.0000,0.0,14.67,4.576562,...,3.283983,13.7300,18.25,0.00,8.32,3.259729,3.11,1.640243,2.0500,4.3225
85,19.75,45.50,34.322750,35.25,4.752477,31.00,38.0000,0.0,13.47,4.456333,...,3.119856,13.5000,17.75,0.00,9.67,3.432562,3.20,1.732727,2.1575,4.5650
86,19.50,46.00,34.546229,35.25,4.842294,31.25,37.8125,0.0,12.47,4.371958,...,2.823124,14.0000,17.75,0.00,10.00,3.338125,3.08,1.656742,2.1600,4.3350
87,23.50,46.25,34.873229,35.25,4.531720,31.75,38.2500,0.0,14.82,4.380583,...,3.131076,13.7500,18.00,0.00,9.51,3.424646,3.27,1.690960,2.1700,4.5000


In [7]:
#part2 - normalized preprecessing

#axis = 1 to normalize each feature
df_idx_norm_array = preprocessing.normalize(df.iloc[:,1:7],axis=1)
df_idx_norm = df

for i in range(42239):
    for j in range(1,7):
        df_idx_norm.iat[i,j] = df_idx_norm_array[i][j-1]

#part2 - normalized
df_idx_norm = df_idx_norm.set_index('instance')

info_df_norm = pd.DataFrame(columns = ['min_1','max_1','mean_1','median_1','std_1','1st_quart_1','3rd_quart_1',
                                 'min_2','max_2','mean_2','median_2','std_2','1st_quart_2','3rd_quart_2',
                                 'min_3','max_3','mean_3','median_3','std_3','1st_quart_3','3rd_quart_3',
                                 'min_4','max_4','mean_4','median_4','std_4','1st_quart_4','3rd_quart_4',
                                 'min_5','max_5','mean_5','median_5','std_5','1st_quart_5','3rd_quart_5',
                                 'min_6','max_6','mean_6','median_6','std_6','1st_quart_6','3rd_quart_6',])

for i in range(1,89):
    row_tmp_norm = []
    for j in range(1,7):
        slice_tmp_norm = [df_idx_norm.loc[i].iloc[:,j].min(), df_idx_norm.loc[i].iloc[:,j].max(),
                          df_idx_norm.loc[i].iloc[:,j].mean(), df_idx_norm.loc[i].iloc[:,j].median(), 
                          df_idx_norm.loc[i].iloc[:,j].std(), 
                          df_idx_norm.loc[i].iloc[:,j].quantile(.25),df_idx_norm.loc[i].iloc[:,j].quantile(.75)]
        row_tmp_norm.extend(slice_tmp_norm)
    info_df_norm.loc[i] = row_tmp_norm
    
info_df_norm

,min_1,max_1,mean_1,median_1,std_1,1st_quart_1,3rd_quart_1,min_2,max_2,mean_2,...,std_5,1st_quart_5,3rd_quart_5,min_6,max_6,mean_6,median_6,std_6,1st_quart_6,3rd_quart_6
1,0.641312,0.784123,0.716733,0.719763,0.029022,0.698780,0.738124,0.0,0.022696,0.006362,...,0.037840,0.592935,0.633103,0.000000,0.035518,0.010025,0.007799,0.010269,0.000000,0.021856
2,0.661738,0.811324,0.740397,0.734299,0.026186,0.721188,0.757747,0.0,0.022132,0.006481,...,0.030199,0.553784,0.591471,0.000000,0.058461,0.009887,0.007728,0.010416,0.000000,0.022065
3,0.642306,0.806053,0.723129,0.723015,0.018166,0.713118,0.733099,0.0,0.030510,0.007103,...,0.023590,0.575935,0.602612,0.000000,0.029520,0.007997,0.007471,0.008291,0.000000,0.015105
4,0.615210,0.824067,0.722142,0.724013,0.038095,0.707361,0.747106,0.0,0.060758,0.012340,...,0.038526,0.552910,0.597576,0.000000,0.050670,0.010487,0.008536,0.009007,0.000000,0.016688
5,0.684126,0.825041,0.761174,0.760616,0.024758,0.747847,0.775718,0.0,0.054325,0.010023,...,0.024835,0.525497,0.559257,0.000000,0.033605,0.007023,0.007811,0.007073,0.000000,0.009541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,0.608216,0.925236,0.813489,0.823330,0.053329,0.778527,0.851437,0.0,0.410201,0.112478,...,0.083789,0.334087,0.441325,0.000000,0.224815,0.080068,0.074480,0.041604,0.048880,0.105096
85,0.639002,0.950316,0.822602,0.829697,0.053898,0.790097,0.863047,0.0,0.370272,0.109117,...,0.077204,0.321097,0.428155,0.000000,0.223197,0.083572,0.076356,0.043278,0.050225,0.112704
86,0.559898,0.932626,0.823830,0.831821,0.052931,0.799901,0.859427,0.0,0.350090,0.105997,...,0.076507,0.328761,0.425930,0.000000,0.243504,0.080623,0.074703,0.040977,0.051315,0.102598
87,0.662072,0.947159,0.823767,0.831901,0.050465,0.794635,0.859975,0.0,0.364251,0.105178,...,0.077952,0.326134,0.427311,0.000000,0.249944,0.081771,0.077991,0.041546,0.051312,0.105961


#### 1(c)iii  Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature.

In [8]:
#std of raw features
std_summary = pd.DataFrame(info_df.std(),columns=['Estimated std'])

#std of normalized features
std_summary_norm = pd.DataFrame(info_df_norm.std(),columns=['Estimated std'])

#bootstrap for raw features
#BootstrapResults: value (lower_bound, upper_bound)
value, lb, ub = [], [], []
for i in range(42):
    feature = info_df.iloc[:,i]
    samples = np.random.choice(feature,feature.shape[0],replace=True)
    bsr = bs.bootstrap(samples, stat_func=bs_stats.std, alpha=0.1)
    value.append(bsr.value)
    lb.append(bsr.lower_bound)
    ub.append(bsr.upper_bound)
    
std_summary['bootstrapped std'] = value
std_summary['90% C.I. lower bound'] = lb
std_summary['90% C.I. upper bound'] = ub

#bootstrap for normalized features
#BootstrapResults: value (lower_bound, upper_bound)
value_norm, lb_norm, ub_norm = [], [], []
for i in range(42):
    feature = info_df_norm.iloc[:,i]
    samples = np.random.choice(feature,feature.shape[0],replace=True)
    bsr = bs.bootstrap(samples, stat_func=bs_stats.std, alpha=0.1)
    value_norm.append(bsr.value)
    lb_norm.append(bsr.lower_bound)
    ub_norm.append(bsr.upper_bound)
    
std_summary_norm['bootstrapped std'] = value_norm
std_summary_norm['90% C.I. lower bound'] = lb_norm
std_summary_norm['90% C.I. upper bound'] = ub_norm

In [9]:
std_summary

,Estimated std,bootstrapped std,90% C.I. lower bound,90% C.I. upper bound
min_1,9.569975,9.742072,8.764057,10.794386
max_1,4.394362,4.593384,3.846260,5.479695
mean_1,5.335718,5.293769,4.740267,5.918110
median_1,5.440054,5.292018,4.785634,5.883281
std_1,1.772153,1.864716,1.672930,2.088269
1st_quart_1,6.153590,6.214777,5.718254,6.808651
3rd_quart_1,5.138925,4.671817,4.038461,5.385379
min_2,0.000000,0.000000,0.000000,0.000000
max_2,5.062729,5.035972,4.748145,5.399492
mean_2,1.574164,1.597907,1.473323,1.766935


In [10]:
std_summary_norm

,Estimated std,bootstrapped std,90% C.I. lower bound,90% C.I. upper bound
min_1,0.160715,0.184837,0.145663,0.229064
max_1,0.046765,0.051041,0.042734,0.061345
mean_1,0.067151,0.060948,0.054812,0.068049
median_1,0.065818,0.062601,0.055755,0.070572
std_1,0.017142,0.015104,0.013606,0.016815
1st_quart_1,0.073183,0.077180,0.067075,0.088002
3rd_quart_1,0.061195,0.058947,0.051640,0.067663
min_2,0.000000,0.000000,0.000000,0.000000
max_2,0.130569,0.119616,0.107859,0.134724
mean_2,0.038072,0.037238,0.034096,0.041467


#### 1(c)iv Use your judgement to select the three most important time-domain features (one option may be min, mean, and max).

Since we are classifying the activities of humans based on time series:

- (min, max）pair and (1st quart, 3rd quart) pair both serve a purpose to tell us the range of a motion. 
    - We don't know if above datasets are obtained "during" a subject's activity or they include "start & end points" of such activity. 
    - (1st quart, 3rd quart) seems to be a better pair for either situation from this perspective.
    - Std summarys of 1(c)iii also say the same. 

- std could tell moving apart from still 
    - E.g. lying/stting should have rather lower std.
    - However, based to std summary from 1c(iii), they don't tell us that much as we expected.

- mean & median function similarly from this classifying perspective
    - But go with mean because it bears more statistical meanings.
    
- things we may miss
    - i.e. cycling and walking are cycled activities and this characteristic should have reflection on data.

Overall, go with (1st quart, 3rd quart, mean) as the triple.

#### 2 ISLR 3.7.4: I collect a set of data (n = 100 observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e. Y = β0 +β1X +β2X2 +β3X3 +ε.


#### (a) Suppose that the true relationship between X and Y is linear, i.e. Y = β0 + β1X + ε. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

We would expect the RSS of cubic regression to be lower than the linear one, since extra parameters allow it to fit tighter.


#### (b) Answer (a) using test rather than training RSS.

When it comes to testing, RSS of linear regression is expected to be lower because it is closer to the true relationship between X and Y. Cubic one bears risk of overfitting under this case.

#### (c) Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

Still expect a lower RSS from cubic regression due to the same reason to (a).


#### (d) Answer (c) using test rather than training RSS.

There is not enough information. It depends on which one is closer to the true relationship, results may vary. RSS is expected to be lower from the closer one.
